In [4]:
import torch
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
from train import SimCLR_Train
from models import SimCLR_Model

In [5]:
# credit: https://github.com/kuangliu/pytorch-cifar/issues/19
# for CIFAR-10 normalization values
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

batch_size_list = [256, 512, 1024, 2048, 4096]

# the maximum batchsize is 1024
batch_size = batch_size_list[2]

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
resnet20 = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=False)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = SimCLR_Model(base_encoder=resnet20)
net = net.to(device)

Using cache found in C:\Users\Koir/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master


In [7]:
# use adam/sgd since we don't have large batches
# possible config for hyperparams:
# temp = [0.1, 0.5, 1.0]
# lr = 1e-4
# epoch = [100, 200, 300]
# could add a cosine scheduler
# check here for more config: https://github.com/p3i0t/SimCLR-CIFAR10
train_loss_list = SimCLR_Train(net, device, batch_size, train_loader, num_epoch=50, temp=0.5, lr=1e-4, optim='adam', lr_scheduler='None')

Epoch 1


100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [01:39<00:00,  2.04s/it]


Epoch: 1, Training Loss: 7650782.0
Epoch 2


100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [01:44<00:00,  2.14s/it]


Epoch: 2, Training Loss: 7287728.0
Epoch 3


...

Epoch: 48, Training Loss: 6780739.0
Epoch 49


100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [01:42<00:00,  2.09s/it]


Epoch: 49, Training Loss: 6779444.0
Epoch 50


100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [01:43<00:00,  2.10s/it]

Epoch: 50, Training Loss: 6782586.0
Finish training, saving model...
